In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
data = pd.read_excel("HCP_Data_KDAG_Hackathon/HCP_Data_KDAG_Hackathon.xlsx")

In [4]:
data.head()

,Physician_ID,Time_Period,Brand_Rx,Market_Rx,Sales_Rep_Calls,Samples_Dropped,Physician_Segment,Emails_Delivered,Speaker_Programs_Attended,Vouchers_Dropped,Specialty
0,axt00001,20190104,0.0,2.80567,1,0,3-Low,0,0,0,Dermatologist
1,axt00001,20190111,0.0,20.57312,1,0,3-Low,0,0,0,Dermatologist
2,axt00001,20190118,0.0,6.16010,1,0,3-Low,0,0,0,Dermatologist
3,axt00001,20190125,0.0,8.95501,1,5,3-Low,0,0,0,Dermatologist
4,axt00001,20190201,0.0,9.13793,1,0,3-Low,0,0,0,Dermatologist


In [19]:
data['Physician_Segment'].value_counts()

3-Low       252966
2-Medium    187302
1-High      129732
Name: Physician_Segment, dtype: int64

In [6]:
len(data[(data['Speaker_Programs_Attended'] > 0) | (data['Vouchers_Dropped']>0)])/len(data)

0.017659649122807018

In [18]:
d = data[data['Specialty'] == 'Nurse Practitioner']
d[d['Emails_Delivered']>0]

,Physician_ID,Time_Period,Brand_Rx,Market_Rx,Sales_Rep_Calls,Samples_Dropped,Physician_Segment,Emails_Delivered,Speaker_Programs_Attended,Vouchers_Dropped,Specialty
2741,axt00049,20190208,0.000,2.69269,0,0,3-Low,1,0,0,Nurse Practitioner
2742,axt00049,20190215,1.882,20.48435,1,0,3-Low,1,0,0,Nurse Practitioner
2745,axt00049,20190308,0.000,7.41902,1,0,3-Low,2,0,0,Nurse Practitioner
2746,axt00049,20190315,1.008,5.05989,1,0,3-Low,2,0,0,Nurse Practitioner
2747,axt00049,20190322,0.000,14.18437,0,0,3-Low,1,0,0,Nurse Practitioner
...,...,...,...,...,...,...,...,...,...,...,...
566315,axt09936,20190524,0.000,0.00000,0,0,3-Low,1,0,0,Nurse Practitioner
566316,axt09936,20190531,0.000,0.00000,0,0,3-Low,1,0,0,Nurse Practitioner
566326,axt09936,20190809,0.000,0.00000,1,0,3-Low,2,0,0,Nurse Practitioner
566331,axt09936,20190913,0.000,0.00000,0,0,3-Low,1,0,0,Nurse Practitioner


In [ ]:
df = data[data['Physician_ID'] == 'axt00001']

In [11]:
df = df.drop(['Physician_ID', 'Time_Period', 'Physician_Segment', 'Speaker_Programs_Attended', 'Vouchers_Dropped', 'Specialty'], axis = 1)

In [17]:
df.head()

,Brand_Rx,Market_Rx,Sales_Rep_Calls,Samples_Dropped,Emails_Delivered
0,0.0,2.80567,1,0,0
1,0.0,20.57312,1,0,0
2,0.0,6.16010,1,0,0
3,0.0,8.95501,1,5,0
4,0.0,9.13793,1,0,0


In [15]:
# xgboost regressor
from xgboost import XGBRegressor

In [34]:
regressor = XGBRegressor()

In [35]:
X = df.drop('Brand_Rx', axis = 1)
y = df['Brand_Rx']

In [36]:
regressor.fit(X, y)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.300000012, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=6, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [51]:
regressor.feature_names_in_

array(['Market_Rx', 'Sales_Rep_Calls', 'Samples_Dropped',
       'Emails_Delivered'], dtype='<U16')

In [48]:
regressor.feature_importances_


array([0.254423  , 0.09616681, 0.11811881, 0.5312914 ], dtype=float32)

In [45]:
# lgbm regressor
import lightgbm as lgb


In [46]:
lgbm = lgb.LGBMRegressor()
lgbm.fit(X, y)

LGBMRegressor()

In [63]:
lgbm.feature_importances_

array([67, 15,  0, 18], dtype=int32)

In [62]:
# get feature importance
lgbm.feature_importances_[1: ].argmax()

2

In [57]:
lgbm.feature_name_

['Market_Rx', 'Sales_Rep_Calls', 'Samples_Dropped', 'Emails_Delivered']

In [79]:
pid = data['Physician_ID'].unique()

In [80]:
pid.sort()

In [81]:
promotion = []
for id in pid:
    df = data[data['Physician_ID'] == id]
    df = df.drop(['Physician_ID', 'Time_Period', 'Physician_Segment', 'Speaker_Programs_Attended', 'Vouchers_Dropped', 'Specialty', 'Market_Rx'], axis = 1)
    X = df.drop('Brand_Rx', axis = 1)
    y = df['Brand_Rx']
    lgbm = lgb.LGBMRegressor()
    lgbm.fit(X, y)
    promotion.append(lgbm.feature_importances_.argmax())

In [82]:
res = pd.DataFrame({'Physician_ID': pid, 'Channel': promotion})
res.head()

,Physician_ID,Channel
0,axt00001,0
1,axt00002,0
2,axt00003,0
3,axt00004,0
4,axt00005,0


In [83]:
res.to_csv('submission2.csv', index = False)